In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2DTranspose, Conv2D, LeakyReLU, Dropout
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt


def makeGenerator():
    return Sequential([
        Dense(7 * 7 * 256, input_dim=100),
        LeakyReLU(alpha=0.2),
        Reshape((7, 7, 256)),
        Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2DTranspose(1, kernel_size=4, strides=1, padding="same", activation="tanh")
    ])


def makeDiscriminator():
    return Sequential([
        Conv2D(64, kernel_size=4, strides=2, padding="same", input_shape=(28, 28, 1)),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Conv2D(128, kernel_size=4, strides=2, padding="same"),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Flatten(),
        Dense(1, activation="sigmoid")
    ])


class GAN:
	def __init__(self, generator, discriminator):
		self.generator = generator
		self.discriminator = discriminator
		gan = Sequential([generator, discriminator])
		gan.compile(loss="binary_crossentropy", 
			optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002))
		self.gan = gan

	def train(self, X_train, epochs=10000, batch_size=128, latent_dim=100):
		half_batch = batch_size // 2
		for epoch in range(epochs):
		    # Train Discriminator
		    idx = np.random.randint(0, X_train.shape[0], half_batch)
		    real_imgs = X_train[idx]

		    noise = np.random.normal(0, 1, (half_batch, latent_dim))
		    fake_imgs = self.generator.predict(noise)

		    real_labels = np.ones((half_batch, 1))
		    fake_labels = np.zeros((half_batch, 1))

		    self.discriminator.trainable = True
		    d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
		    d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
		    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

		    # Train Generator
		    noise = np.random.normal(0, 1, (batch_size, latent_dim))
		    misleading_labels = np.ones((batch_size, 1))

		    self.discriminator.trainable = False
		    g_loss = self.gan.train_on_batch(noise, misleading_labels)

		    # Print progress
		    if epoch % 1000 == 0:
		        print(f"{epoch}: [D loss: {d_loss[0]} | acc: {d_loss[1]*100}] [G loss: {g_loss}]")


def generate_images(generator, num_images=10):
    noise = np.random.normal(0, 1, (num_images, 100))
    generated_images = generator.predict(noise)

    fig, axes = plt.subplots(1, num_images, figsize=(10, 2))
    for i in range(num_images):
        axes[i].imshow(generated_images[i].reshape(28, 28), cmap="gray")
        axes[i].axis("off")
    plt.show()


discriminator = makeDiscriminator()
discriminator.compile(loss="binary_crossentropy", 
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002), metrics=["accuracy"])	

# don't compile generator. it is trained as part of the gan.
generator = makeGenerator()

(X_train, _), _ = tf.keras.datasets.mnist.load_data()
X_train = (X_train - 127.5) / 127.5  # Normalize to [-1, 1]
X_train = np.expand_dims(X_train, axis=-1)  # Reshape to (28, 28, 1)

gan = GAN(generator, discriminator)
gan.train(X_train)

generate_images(generator)

2025-03-30 09:13:09.518201: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-30 09:13:09.569657: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-30 09:13:09.569705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-30 09:13:09.571149: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-30 09:13:09.578699: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-30 09:13:09.579248: I tensorflow/core/platform/cpu_feature_guard.cc:1

2/2 [==============================] - 0s 93ms/step
0: [D loss: 0.6983038187026978 | acc: 21.09375] [G loss: 0.6915892362594604]
2/2 [==============================] - 0s 28ms/step


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7fc955b076a0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/weakref.py", line 369, in remove
    def remove(k, selfref=ref(self)):

KeyboardInterrupt: 


KeyboardInterrupt: 